# Cluster Analysis

Cluster analysis is a multivariate statistical technique that groups observations on the basis some of their features or variables they are described by.

In this notebook we will explore:

    1. Several Clustering Problems
    2. How to perform cluster analysis
    3. How to find the optimal number of clusters
    4. How to identify appropriate features
    5. How to interpret the results

### Math Pre-requisites

In order to study and apply efficiently the K-means clustering method, we need to understand its underlying procedure. In other words, we must understand the mathematical approach behind the technique. We will define two concepts here, the **euclidian distance** and the concept of **centroid**.

______________
#### Euclidean distance

the Euclidean distance between two points $x$ and $y$ in a bidimensional space is the **length of the line** connecting them.

<img src='images/euclidean.png'>

As you can see from the figure, we can draw a triangle in the plane with the coordinate of the points. Consequently, the distance is given by the Pythagoras' theorem. Therefore,

$$d(x,y)^2 = (y_1 - x_1)^2 + (y_2 - x_2)^2$$ 

where $x = (x_1,x_2)$ and $y = (y_1,y_2)$ are the coordinates of the points $x$ and $y$, respectively. The equation can be written in the following way

$$d(x,y) = \sqrt{(y_1 - x_1)^2 + (y_2 - x_2)^2}$$ 

analogously, we can also use this result for n-dimensional spaces. Therefore, for $x = (x_1, x_2, ... , x_n)$ and $y = (y_1, y_2, ... , y_n)$

$$d(x,y) = \sqrt{(y_1 - x_1)^2 + (y_2 - x_2)^2 + ... + (y_n - x_n)^2}$$ 

_____________
#### Centroid

In mathematics and physics, the centroid or geometric center of a plane figure is the **arithmetic mean position** of all the points in the figure. For example, take 3 points, $a = (2, 1)$, $b = (3, 4)$ and $c = (4, 1)$. the centroid, $\alpha$, is given by 

$$\alpha = (\tfrac{2 + 3 + 4}{3},\tfrac{1 + 4 + 1}{3}) = (3, 2)$$

Maybe it will be easier if we visualize the points

<img src='images/centroid.png'>

This is a simple example but we can extend this result to more complex situations and any number of points.

## 1. K-Means Clustering